## Research Data Management for Data Science 
#### UC Berkeley Library, Research IT

* Data Evaluation
* Data Cleaning
    * Reshaping and joining
    * Variable names and types
    * Missing values, nulls, and zeros
* Reproducibility/Metadata 




# Data Evaluation
|                                                                                                                   |
|-------------------------------------------------------------------------------------------------------------------|
| What makes this dataset findable?                                                                                 |
| Can you tell if this is original/raw data or if it's been manipulated? Who created this?                          |
| Can you tell what the unit of observation is?                                                                     |
| If you had to merge or add to this, could you?                                                                    |
| Do you think this data can answer our question _What is the proportion of women to men in the Technology Industry?_ |
|                                                                                                                   |

|         |                                                                       |
|---------|-----------------------------------------------------------------------|
| Group 1 | https://www.eeoc.gov/eeoc/statistics/reports/hightech/                |
| Group 2 | https://github.com/alison985/women-in-tech-datasets                   |
| Group 3 | https://www.dol.gov/wb/stats/Computer_information_technology_2014.htm |
| Group 4 | https://www.bls.gov/cps/cpsaat11.htm                                  |
| Group 5 | https://berkeley.box.com/s/ik0obara8hj8k212logii0sh9m5vwj63           |

# Data Evaluation


![](./images/quality.png)

[Quartz Guide to Bad Data](https://qz.com/572338/the-quartz-guide-to-bad-data/)




# Data Cleaning

> “Happy families are all alike; every unhappy family is unhappy in its own way.” 

– Leo Tolstoy


> “Tidy datasets are all alike, but every messy dataset is messy in its own way.”

– Hadley Wickham 



There is a much repeated saying in data science that 80% of data analysis is applying cleaning operations to get the dataset to a usable form for analysis. Because transforming a raw data set is time consuming and will take several iterations, it is important to think about the lifecycle of a dataset, how it changes, and how you will document those changes to others. 

We will look at some of the more common techniques for [Tidy Data]("http://r4ds.had.co.nz/tidy-data.html").



Tidy Data has the following attributes:

* Each variable forms a column and contains values
* Each observation forms a row

  
 ![](./images/tidy.png)





### Reshaping


In [1]:
import pandas as pd

df = pd.read_csv("./data/pew-raw.csv")
df

,religion,$0-$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


df is a common looking data table designed to be easily readable in print. Many times you will find data like this if it has been digitized from government documents or survey providers (like Pew or Gallup) and provided in Excel spreadsheets. Though it is human readable it is not useful for analysis. 

We want to [melt]("http://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html") this data from *wide* to *long* so that income classes are no longer in the column header and the count values are no longer spread out through the table but neatly contained as variables in columns.

In [2]:
tidy_df = pd.melt(df,
                  ["religion"],
                  var_name="income",
                  value_name="n")
tidy_df = tidy_df.sort_values(by=["religion"])
tidy_df.head(10)




,religion,income,n
0,Agnostic,$0-$10k,27
30,Agnostic,$30-40k,81
40,Agnostic,$40-50k,76
50,Agnostic,$50-75k,137
10,Agnostic,$10-20k,34
20,Agnostic,$20-30k,60
41,Atheist,$40-50k,35
21,Atheist,$20-30k,37
11,Atheist,$10-20k,27
31,Atheist,$30-40k,52


If we want to do the reverse and revert it to a "pivot-table" like dataset as we saw before, the [pivot]("https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html") function can be used.

In [3]:
wide_df = tidy_df.pivot( 
                   index = 'religion', 
                   columns = 'income', 
                   values = 'n')
wide_df.head()

income,$0-$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
religion,,,,,,
Agnostic,27,34,60,81,76,137
Atheist,12,27,37,52,35,70
Buddhist,27,21,30,34,33,58
Catholic,418,617,732,670,638,1116
Dont know/refused,15,14,15,11,10,35


In [17]:
# split variable
tidy_df['lower_inc'] = tidy_df.income.str.split('-').str.get(0)
tidy_df['upper_inc'] = tidy_df.income.str.split('-').str.get(1) 

tidy_df.head()



,religion,income,n,lower_inc,upper_inc
0,Agnostic,$0-$10k,27,$0,$10k
30,Agnostic,$30-40k,81,$30,40k
40,Agnostic,$40-50k,76,$40,50k
50,Agnostic,$50-75k,137,$50,75k
10,Agnostic,$10-20k,34,$10,20k


In [16]:
print(tidy_df.describe())


                 n
count    60.000000
mean    213.250000
std     344.877902
min       1.000000
25%      21.000000
50%      34.000000
75%     230.000000
max    1486.000000


## Merging



 ![](./images/sql-joins.png)


In [6]:
members = pd.read_csv("./data/members.csv")
instruments = pd.read_csv("./data/instruments.csv")

In [7]:
from IPython.display import display

display(members.head())
display(instruments.head())


,name,band
0,Mick,Stones
1,John,Beatles
2,Paul,Beatles


,name,plays
0,John,guitar
1,Paul,bass
2,Keith,guitar


In [20]:
pd.merge(members, instruments, how = 'right', on = 'name')

,name,band,plays
0,John,Beatles,guitar
1,Paul,Beatles,bass
2,Keith,NaN,guitar


Left merge will merge keys from "left" dataframe

Right merge will merge keys from "right" dataframe. 


In [9]:
pd.merge(members, instruments, how = 'outer', on = 'name')

,name,band,plays
0,Mick,Stones,NaN
1,John,Beatles,guitar
2,Paul,Beatles,bass
3,Keith,NaN,guitar


Inner joins retain only the rows common to both datasets. Try changing the *how* argument to run an outer join. What happens?

## Text Data

In [22]:
tidy_df['lower_inc'] = tidy_df.lower_inc.str.strip("$")

tidy_df.head()

,religion,income,n,lower_inc,upper_inc
0,Agnostic,$0-$10k,27,0,$10k
30,Agnostic,$30-40k,81,30,40k
40,Agnostic,$40-50k,76,40,50k
50,Agnostic,$50-75k,137,50,75k
10,Agnostic,$10-20k,34,10,20k


## Text Data

Converting the entire document to lower case

Removing punctuation marks (periods, commas, hyphens etc)

Removing stopwords (extremely common words such as “and”, “or”, “not”, etc)

Removing numbers

Filtering out unwanted terms

Removing extra whitespace





## Text Data

[Regular Expressions in Python](https://github.com/dlab-berkeley/regular-expressions-in-python)

[Regexone](https://regexone.com/references/python)

[Learn Regex the Easy Way](https://github.com/zeeshanu/learn-regex)

## Variable Management

In [25]:
for col in tidy_df:
    print (type(tidy_df[col][1]))
    
tidy_df.head()

#change lower_inc to numeric
#tidy_df["lower_inc"] = pd.to_numeric(tidy_df["lower_inc"])


<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'str'>


,religion,income,n,lower_inc,upper_inc
0,Agnostic,$0-$10k,27,0,$10k
30,Agnostic,$30-40k,81,30,40k
40,Agnostic,$40-50k,76,40,50k
50,Agnostic,$50-75k,137,50,75k
10,Agnostic,$10-20k,34,10,20k


### Dates



In [12]:
dates = pd.read_csv("./data/dates.csv")
dates

,month_day_year,day_month_year,date_time,year_month_day
0,4/22/1996,22-Apr-96,Tue Aug 11 09:50:35 1996,2007-06-22
1,4/23/1996,23-Apr-96,Tue May 12 19:50:35 2016,2017-01-09
2,5/14/1996,14-May-96,Mon Oct 14 09:50:35 2017,1998-04-12
3,5/15/1996,15-May-96,Tue Jan 11 09:50:35 2018,2027-07-22
4,5/16/2001,16-May-01,Fri Mar 11 07:30:36 2019,1945-11-15
5,5/17/2002,17-May-02,Tue Aug 11 09:50:35 2020,1942-06-22
6,5/18/2003,18-May-03,Wed Dec 21 09:50:35 2021,1887-06-13
7,5/19/2004,19-May-04,Tue Jan 11 09:50:35 2022,1912-01-25
8,5/20/2005,20-May-05,Sun Jul 10 19:40:25 2023,2007-06-22


In [13]:
#check data types
for col in dates:
    print (type(dates[col][1]))



<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [14]:
#convert str to date
dates["date_time"] = pd.to_datetime(dates["date_time"] )

dates

,month_day_year,day_month_year,date_time,year_month_day
0,4/22/1996,22-Apr-96,1996-08-11 09:50:35,2007-06-22
1,4/23/1996,23-Apr-96,2016-05-12 19:50:35,2017-01-09
2,5/14/1996,14-May-96,2017-10-14 09:50:35,1998-04-12
3,5/15/1996,15-May-96,2018-01-11 09:50:35,2027-07-22
4,5/16/2001,16-May-01,2019-03-11 07:30:36,1945-11-15
5,5/17/2002,17-May-02,2020-08-11 09:50:35,1942-06-22
6,5/18/2003,18-May-03,2021-12-21 09:50:35,1887-06-13
7,5/19/2004,19-May-04,2022-01-11 09:50:35,1912-01-25
8,5/20/2005,20-May-05,2023-07-10 19:40:25,2007-06-22


## Variable Naming Best Practices




|Good Example   | Bad Example   | Description  |
|---|---|---|
|gnp2010   |gnp-2002; gnp#2002    |   |
|real_int    |real interest rate    |   |
| score1; gnp2003   | 1st_score; 2003gnp  |  |
|reg_out; glm1    | REG; glm; ttest   |   |
| invest; interest  | xxx; yyy; zmdje;   |    |
|male; asian    | gender; race   |   |
| citizen   | Are_you_a_US_citizen?   |   |
| income; intUS03   | INCOME; Int_us2003;   |   |
| 2017-04-20   |April 20, 2017   |   |   |


## Variable Naming Best Practices



|Good Example   | Bad Example   | Description  |
|---|---|---|
|gnp2010   |gnp-2002; gnp#2002    | avoid special characters  |
|real_int    |real interest rate    |Use underscore   |
| score1; gnp2003   | 1st_score; 2003gnp  | Begin with a character   |
|reg_out; glm1    | REG; glm; ttest   | Avoid reserved words  |
| invest; interest  | xxx; yyy; zmdje;   |Use meaningful names    |
|male; asian    | gender; race   | Use a value of dummy   |
| fav_color   | Whats_Your_Favorite_Color?   | The shorter, the better   |
| income; intUS03   | INCOME; Int_us2003;   | Use lower cases   |
| 2017-04-20   |April 20, 2017   | Use common ISO year format  |


## Missing Values, Nulls, and Zeros

![](images/nulls.png)

In [15]:
tidy_df.dropna()
#tidydf.fillna(value)



,religion,income,n,lower_inc,upper_inc
0,Agnostic,$0-$10k,27,0,$10k
30,Agnostic,$30-40k,81,30,40k
40,Agnostic,$40-50k,76,40,50k
50,Agnostic,$50-75k,137,50,75k
10,Agnostic,$10-20k,34,10,20k
20,Agnostic,$20-30k,60,20,30k
41,Atheist,$40-50k,35,40,50k
21,Atheist,$20-30k,37,20,30k
11,Atheist,$10-20k,27,10,20k
31,Atheist,$30-40k,52,30,40k


Tidy Data has the following attributes:

* Each variable forms a column and contains values
* Each observation forms a row

  
 ![](./images/tidy.png)


Tidy _and useful_ Data has the following attributes:

* Each variable forms a column and contains values
* Each observation forms a row
* Data types = What you expect them to be 
* [Appropriate observations and variables to address a question](https://www.theatlantic.com/business/archive/2013/04/forget-excel-this-was-reinhart-and-rogoffs-biggest-mistake/275088/)
* Missing values are accounted for, Zeros really zero

### Well-Described Data...

* Precisely IDs the dataset
* Provides details of its origin and history
* Eliminates confusion (and error)
* Includes “intrinsic metadata”
* Justifies every decision made re: the handling of the data
* Allows discovery and reuse




["Science is show me, not trust me"](http://www.bitss.org/2015/12/31/science-is-show-me-not-trust-me)

[Findability, Accessibiliy, Interoperability, Reusability (F.A.I.R) Principles](https://www.force11.org/group/fairgroup/fairprinciples)

## Reaching out

Rick Jaffe, rjaffe@berkeley.edu

Josh Quan, joshua.quan@berkeley.edu

